# -----Importing the libraries ----#

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import numpy as np
from matplotlib import style
import datetime as dt
from functools import reduce 
import time
import pandas as pd
from IPython.display import clear_output
import re
import ta
from scipy.signal import argrelextrema
from scipy.signal import find_peaks, peak_prominences
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing
from scipy.spatial.distance import euclidean
# from fastdtw import fastdtw
from ta import *
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import calendar
from dateutil import relativedelta
import warnings
warnings.filterwarnings("ignore")
import itertools


from datetime import date, timedelta as td

from functools import reduce
import inspect
from functools import partial
from multiprocessing import Pool
# import talib

# ------ Rules ----------- #
Bulls_strategy

1. The close must be above the 200-day average.
2. The 5-day RSI indicator of the 5-day Rate of Change indicator must be below 20.
3. If 1 and 2 are true, then buy the close.
4. We sell at the close when the 5-day RSI is above 50.

In [4]:
strategy_name = 'Bulls_strategy' # ----------- Edit the Strategy_name ------------- #

In [5]:
# ---------- Paths -----------#

#-------------- export path -----------#

initial_export_path = "C:/Users/user/OneDrive/WSG Markets/CMC Backtesting till Jan 2025 (5 min)/BT results/" + strategy_name + "/"


export_path_Open_Trades =  initial_export_path + "/Open_Trades/"
export_path_trade_level_path =  initial_export_path + "/Trade Level Data/"
export_path_BT =  initial_export_path + "/BT/"
export_path_FT =  initial_export_path + "/FT/"
path_for_error_files = initial_export_path + "/Error/"

try:
    os.mkdir(initial_export_path)
    os.mkdir(export_path_Open_Trades)
    os.mkdir(export_path_trade_level_path)
    os.mkdir(export_path_BT)
    os.mkdir(export_path_FT)
    os.mkdir(path_for_error_files)

except:
    print("Folder Already Created")

# #-------------- Entry data export path -----------#
# Entry_data_export_path = "E:/WSG Markets/MA-RSI backtesting/Exit by RSI value/Exit Data/"

#-------------- fxcm data export path -----------#
fxcm_data_export_path = "E:/WSG Markets/CMC Backtesting till Jan 2025 (5 min)/Raw data/"

Folder Already Created


# -----Loading the data -----#

In [7]:

# ---------- d1 Data -----------#
d1 = pd.read_parquet(
    fxcm_data_export_path + "CMC_DAILY_OHLC_close_m20_m25_m30_updated.parquet"
)  # ------ it is Fwd looking data ------- #

d1['date'] = pd.to_datetime(d1['date'], format='%Y-%m-%d')
d1.rename(columns={'Symbol': 'Currency'}, inplace=True)
d1['Timeframe'] = "D1"

price_data = pd.concat([d1], axis=0, ignore_index=True)

price_data.tail()

,org_symbol,date,askopen,askhigh,asklow,askclose,bar_ts_close,open_m05,high_m05,low_m05,...,end_m05,end_m10,end_m15,end_m20,bidclose,bidopen,bidhigh,bidlow,Timeframe,Currency
4315612,ZYME,2026-01-15,24.650000,24.799999,23.139999,23.629999,2026-01-15 21:00:00+00:00,24.650000,24.799999,23.139999,...,2026-01-15 20:55:00+00:00,2026-01-15 20:50:00+00:00,2026-01-15 20:45:00+00:00,2026-01-15 20:40:00+00:00,23.629999,24.650000,24.799999,23.139999,D1,ZYM/USD
4315613,ZYME,2026-01-16,23.860001,23.860001,23.139999,23.350000,2026-01-16 21:00:00+00:00,23.860001,23.860001,23.139999,...,2026-01-16 20:55:00+00:00,2026-01-16 20:50:00+00:00,2026-01-16 20:45:00+00:00,2026-01-16 20:40:00+00:00,23.350000,23.860001,23.860001,23.139999,D1,ZYM/USD
4315614,ZYME,2026-01-20,22.940001,24.100000,22.799999,23.670000,2026-01-20 21:00:00+00:00,22.940001,24.100000,22.799999,...,2026-01-20 20:55:00+00:00,2026-01-20 20:50:00+00:00,2026-01-20 20:45:00+00:00,2026-01-20 20:40:00+00:00,23.670000,22.940001,24.100000,22.799999,D1,ZYM/USD
4315615,ZYME,2026-01-21,24.014999,24.170000,23.240000,23.950001,2026-01-21 21:00:00+00:00,24.014999,24.170000,23.240000,...,2026-01-21 20:55:00+00:00,2026-01-21 20:50:00+00:00,2026-01-21 20:45:00+00:00,2026-01-21 20:40:00+00:00,23.950001,24.014999,24.170000,23.240000,D1,ZYM/USD
4315616,ZYME,2026-01-22,23.950001,24.309999,23.500000,23.549999,2026-01-22 21:00:00+00:00,23.950001,24.309999,23.500000,...,2026-01-22 20:55:00+00:00,2026-01-22 20:50:00+00:00,2026-01-22 20:45:00+00:00,2026-01-22 20:40:00+00:00,23.549999,23.950001,24.309999,23.500000,D1,ZYM/USD


In [8]:
# optional (faster + less memory)
price_data["Timeframe"] = price_data["Timeframe"].astype("category")
price_data["Currency"]  = price_data["Currency"].astype("category")

# Build lookup ONCE:
PRICE_BY_KEY = {
    (tf, cur): g
    for (tf, cur), g in price_data.groupby(["Timeframe", "Currency"], sort=False)
}

# ---------------------- Combinations Iteration List ----------------------- #

In [10]:
strategy = [strategy_name]

# ✅ All symbols will be processed together inside Func_param()
symbols = price_data['Currency'].unique()

symbols_not_to_BT = ["XTI/USD","XNG/USD","USD/TRY","EUR/TRY","USD/CNH","TRY/JPY"]
symbols = list(set(symbols) - set(symbols_not_to_BT))

condition = ["Long"]
tf = ["D1"]

# -------- Choose the parameter as per strategy -------------- #
parameter_1 = list(range(180,220,10)) # n Moving average
parameter_2 = list(range(4,11,2))     # ROC & RSI of ROC
parameter_3 = list(range(15,30,5))    # threshold for RSI of ROC.
parameter_4 = list(range(50,100,10))  # threshold for RSI of ROC exit
parameter_5 = list(range(10,11,1))    # parameter not required, keeping it constant.
parameter_6 = list(range(10,11,1))    # parameter not required, keeping it constant.

# ✅ PARAM-LEVEL iteration list (Currency removed)
all_param_combinations = itertools.product(
    strategy, condition, tf,
    parameter_1, parameter_2, parameter_3,
    parameter_4, parameter_5, parameter_6
)

iter_list = pd.DataFrame(
    all_param_combinations,
    columns=[
        "Strategy","Condition","Timeframe",
        "parameter_1","parameter_2","parameter_3","parameter_4","parameter_5","parameter_6"
    ]
)

iter_list


In [11]:
iter_list

,Strategy,Currency,Condition,Timeframe,parameter_1,parameter_2,parameter_3,parameter_4,parameter_5,parameter_6,Quote_Curr
0,Bulls_strategy,AA0/USD,Long,D1,180,4,15,50,10,10,USD
1,Bulls_strategy,AA0/USD,Long,D1,180,4,15,70,10,10,USD
2,Bulls_strategy,AA0/USD,Long,D1,180,4,15,90,10,10,USD
3,Bulls_strategy,AA0/USD,Long,D1,180,4,20,60,10,10,USD
4,Bulls_strategy,AA0/USD,Long,D1,180,4,20,80,10,10,USD
...,...,...,...,...,...,...,...,...,...,...,...
214675,Bulls_strategy,ZYM/USD,Long,D1,210,10,20,50,10,10,USD
214676,Bulls_strategy,ZYM/USD,Long,D1,210,10,20,70,10,10,USD
214677,Bulls_strategy,ZYM/USD,Long,D1,210,10,20,90,10,10,USD
214678,Bulls_strategy,ZYM/USD,Long,D1,210,10,25,60,10,10,USD


In [12]:
completed_file_path = export_path_trade_level_path
error_completed_file_path = path_for_error_files

completed_file_1 = os.listdir(completed_file_path)
completed_file_2 = os.listdir(error_completed_file_path)
completed_file_2 = [f.replace("_error", "") for f in completed_file_2]

completed_set = set(completed_file_1) | set(completed_file_2)

# ----------------------------
# Build file_name vectorized + filter (PARAM-LEVEL)
# Naming:
#   ALL_<Timeframe>_<p1>_<p2>_<p3>_<p4>_<p5>_<p6>_<Condition>.parquet
# ----------------------------
file_name = (
    "ALL_" + iter_list["Timeframe"].astype(str) + "_" +
    iter_list["parameter_1"].astype(str) + "_" +
    iter_list["parameter_2"].astype(str) + "_" +
    iter_list["parameter_3"].astype(str) + "_" +
    iter_list["parameter_4"].astype(str) + "_" +
    iter_list["parameter_5"].astype(str) + "_" +
    iter_list["parameter_6"].astype(str) + "_" +
    iter_list["Condition"].astype(str) + ".parquet"
)

mask = ~file_name.isin(completed_set)
iteration_list = iter_list.loc[mask].reset_index(drop=True).values.tolist()

len(iteration_list)


Completed: 0 Pending: 214680


In [13]:
# iteration_list = iteration_list[:1]
iteration_list

[['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 4, 15, 50, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 4, 15, 70, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 4, 15, 90, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 4, 20, 60, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 4, 20, 80, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 4, 25, 50, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 4, 25, 70, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 4, 25, 90, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 6, 15, 60, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 6, 15, 80, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 6, 20, 50, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 6, 20, 70, 10, 10, 'USD'],
 ['Bulls_strategy', 'AA0/USD', 'Long', 'D1', 180, 6, 20, 90, 10, 10, 'USD'],

In [14]:
len(iteration_list)

214680

# ----Main Algo -----#

In [ ]:
def Func_param(it, iteration_list, price_data, initial_export_path, symbols_universe=None):
    # PARAM-LEVEL run:
    #   - Iterates ONLY on (Strategy, Condition, Timeframe, p1..p6)
    #   - Computes signals + trades for ALL symbols in one go
    #   - Exports ONE parquet with all instruments

    import numpy as np
    import pandas as pd
    import pyarrow.parquet as pq
    import pyarrow
    from IPython.display import clear_output

    i = iteration_list[it]
    Strategy    = i[0]
    condition   = i[1]
    timeframe   = i[2]
    parameter_1 = int(i[3])
    parameter_2 = int(i[4])
    parameter_3 = int(i[5])
    parameter_4 = int(i[6])
    parameter_5 = int(i[7])
    parameter_6 = int(i[8])

    # ----------------------------
    # Filter timeframe (+ optional symbol universe)
    # ----------------------------
    df_all = price_data[price_data["Timeframe"] == timeframe].copy()

    if symbols_universe is not None:
        df_all = df_all[df_all["Currency"].isin(symbols_universe)].copy()

    df_all["date"] = pd.to_datetime(df_all["date"])
    df_all = df_all.sort_values(["Currency", "date"]).reset_index(drop=True)

    # Attach params (same for all rows in this run)
    df_all["Strategy"]     = Strategy
    df_all["Condition"]    = condition
    df_all["parameter_1"]  = parameter_1
    df_all["parameter_2"]  = parameter_2
    df_all["parameter_3"]  = parameter_3
    df_all["parameter_4"]  = parameter_4
    df_all["parameter_5"]  = parameter_5
    df_all["parameter_6"]  = parameter_6

    g = df_all.groupby("Currency", sort=False)

    # ----------------------------
    # Vectorized n_MA using (w-1) past askclose + current close_m05
    # ----------------------------
    w = parameter_1
    n = parameter_2

    sum_past = g["askclose"].apply(lambda s: s.shift(1).rolling(window=w-1, min_periods=w-1).sum())
    sum_past = sum_past.reset_index(level=0, drop=True)

    df_all["n_MA"] = (sum_past + df_all["close_m05"]) / w

    # ----------------------------
    # Vectorized ROC: current close_m05 vs lag askclose[n]
    # ----------------------------
    lag = g["askclose"].shift(n)
    df_all["nD_ROC"] = (df_all["close_m05"] - lag) / lag * 100.0

    # ----------------------------
    # RSI of ROC (Wilder-style)
    # ----------------------------
    def rsi_wilder(x: pd.Series, window: int) -> pd.Series:
        delta = x.diff()
        gain = delta.clip(lower=0)
        loss = (-delta).clip(lower=0)
        avg_gain = gain.ewm(alpha=1/window, adjust=False, min_periods=window).mean()
        avg_loss = loss.ewm(alpha=1/window, adjust=False, min_periods=window).mean()
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))

    df_all["nD_RSI_of_ROC"] = g["nD_ROC"].apply(lambda s: rsi_wilder(s, n)).reset_index(level=0, drop=True)

    # ----------------------------
    # Entry / Exit signals
    # ----------------------------
    df_all["Entry_signal"] = np.where(
        (df_all["close_m05"] > df_all["n_MA"]) & (df_all["nD_RSI_of_ROC"] < parameter_3),
        "Long",
        0
    )

    df_all["Trading Price Point"] = np.where(df_all["Entry_signal"] == "Long", df_all["askclose"], 0)
    df_all["TRADE_OPEN_Flag"] = np.where(df_all["Entry_signal"] == "Long", 1, 0)
    df_all["Exit_Signal"] = np.where(df_all["nD_RSI_of_ROC"] > parameter_4, 1, 0)

    # ----------------------------
    # For each ENTRY row: find first EXIT row after it (else use last row),
    # then apply your "if >3 bars then close at entry+3" rule
    # ----------------------------
    df_all["RSI_exit_status"] = np.nan

    exit_cols = [
        ("Exit_askclose", "askclose"),
        ("Exit_askopen", "askopen"),
        ("Exit_asklow", "asklow"),
        ("Exit_askhigh", "askhigh"),
        ("Exit_bidclose", "bidclose"),
        ("Exit_bidopen", "bidopen"),
        ("Exit_bidlow", "bidlow"),
        ("Exit_bidhigh", "bidhigh"),
        ("Closing Date", "date"),
    ]
    for out_col, _ in exit_cols:
        df_all[out_col] = np.nan

    def attach_exits(one: pd.DataFrame) -> pd.DataFrame:
        one = one.reset_index(drop=True)
        nrows = len(one)
        if nrows == 0:
            return one

        entry_idx = np.flatnonzero(one["TRADE_OPEN_Flag"].values == 1)
        if entry_idx.size == 0:
            return one

        exit_idx = np.flatnonzero(one["Exit_Signal"].values == 1)

        last_i = nrows - 1

        pos = np.searchsorted(exit_idx, entry_idx + 1, side="left")
        chosen = np.where(pos < exit_idx.size, exit_idx[pos], last_i)

        cap = entry_idx + 3
        chosen = np.where((chosen - entry_idx) > 3, np.minimum(cap, last_i), chosen)

        for (out_col, src_col) in exit_cols:
            one.loc[entry_idx, out_col] = one.loc[chosen, src_col].values

        one.loc[entry_idx, "RSI_exit_status"] = np.where(chosen == last_i, "RSI_exit_not_reached", "RSI_exit_reached")
        return one

    df_all = g.apply(attach_exits).reset_index(drop=True)

    # ----------------------------
    # Trade-level table (same filter as your original export)
    # ----------------------------
    trade_df = df_all[df_all["RSI_exit_status"] == "RSI_exit_reached"].copy()
    if trade_df.empty:
        trade_df = df_all.head(0).copy()

    trade_df["Total Amount"] = 100000
    trade_df["Risk_Percent"] = 0.01
    trade_df["Units"] = (trade_df["Total Amount"] / trade_df["Trading Price Point"]).round()

    trade_df["Profit_Loss"] = np.where(
        trade_df["Entry_signal"] == "Long",
        (trade_df["Units"] * trade_df["Exit_bidclose"]) - (trade_df["Units"] * trade_df["Trading Price Point"]),
        (trade_df["Units"] * trade_df["Trading Price Point"]) - (trade_df["Units"] * trade_df["Exit_askclose"])
    )
    trade_df["Good_Bad_Flag"] = np.where(trade_df["Profit_Loss"] > 0, 1, 0)

    trade_df["date"] = pd.to_datetime(trade_df["date"])
    trade_df["Year"] = trade_df["date"].dt.year
    trade_df["EntryDate Year"] = trade_df["date"].dt.year
    trade_df["EntryDate Month"] = trade_df["date"].dt.month

    trade_df["Closing Date"] = pd.to_datetime(trade_df["Closing Date"])
    trade_df["ClosingDate Year"] = trade_df["Closing Date"].dt.year
    trade_df["ClosingDate Month"] = trade_df["Closing Date"].dt.month

    # ----------------------------
    # Export ONE combined file for ALL symbols
    # ----------------------------
    export_path_trade_level_path = initial_export_path + "/Trade Level Data/"
    out_name = (
        "ALL_" + str(timeframe) + "_" +
        str(parameter_1) + "_" + str(parameter_2) + "_" + str(parameter_3) + "_" +
        str(parameter_4) + "_" + str(parameter_5) + "_" + str(parameter_6) + "_" +
        str(condition) + ".parquet"
    )

    pq.write_table(pyarrow.table(trade_df), export_path_trade_level_path + out_name, flavor='fastparquet')
    clear_output()


In [ ]:
cpu_count = 2

def parallel_task(func, iteration_list, cpus, price_data, initial_export_path, symbols_universe=None):
    import inspect
    with open(f'./tmp_func_Long.py', 'w') as file:
        file.write(inspect.getsource(func).replace(func.__name__, "task"))

    from tmp_func_Long import task

    if __name__ == '__main__':
        from multiprocessing import Pool
        from functools import partial

        func = partial(
            task,
            iteration_list=iteration_list,
            price_data=price_data,
            initial_export_path=initial_export_path,
            symbols_universe=symbols_universe
        )

        pool = Pool(processes=cpus)
        res = pool.map(func, range(len(iteration_list)))
        pool.close()
        pool.join()
        return res
